In [16]:
import jieba
import jieba.posseg
import numpy as np
import pandas as pd
import random
import math
import operator
from pyecharts import options
from pyecharts.charts import WordCloud
import jieba.analyse

# 预处理

In [17]:
# 获取停用词
def Stop_words():
    stopwords = []
    data = []
    f = open('./stopword.txt', encoding='utf8')
    for line in f.readlines():
        data.append(line)
    for i in data:
        output = str(i).replace('\n', '')
        stopwords.append(output)
    return stopwords
stopwords=Stop_words()
print('停用词表的大小为：',len(stopwords))

停用词表的大小为： 767


In [18]:
# 获取处理文本
text = []
for line in open("corpus4keyword.txt", "r", encoding='utf-8').readlines():  # 将文本文档里面的段落转为字符串
    line = line.replace('\n', '')
    line = line.replace('\ue40c','')
    text.append(line)   
len(text)

164

In [19]:
# 随机抽取100条文本
def create_random(n):
    a = []
    while len(a) != n:
        b = random.randint(0, len(text))
        if b not in a:
            a.append(b)
    return a
text =  [text[i] for i in create_random(100)]
len(text)

100

#  TF-IDF 算法

In [20]:
# 加载文档集，对文档集过滤词性和停用词
def Filter_words(stopwords):
    documents = []
    for line in text:
        segment = jieba.posseg.cut(line.strip())
        filter_words = []
        for word, flag in segment:
            if flag.startswith('n') is False:
                continue
            if word not in stopwords and len(word) > 1:
                filter_words.append(word)
        documents.append(filter_words)
    return documents
data_path = './corpus4keyword.txt'
documents=Filter_words(stopwords)
print('文档的数量为',len(documents))
documents

文档的数量为 100


[['童童',
  '被遗弃',
  '酒店',
  '后门',
  '外表',
  '现实',
  '童童',
  '先天性',
  '心脏病',
  '先天性',
  '肛门',
  '常人',
  '无法',
  '想象',
  '慈善机构',
  '童童',
  '医疗',
  '童童',
  '北京',
  '童童',
  '肛门',
  '手术',
  '康复',
  '情况',
  '思源',
  '焦点',
  '公益',
  '基金',
  '童童',
  '心脏病',
  '手术'],
 ['深圳市',
  '法制办',
  '网站',
  '深圳经济特区',
  '社会',
  '条例',
  '条例',
  '社会',
  '意见',
  '条例',
  '流浪',
  '人员',
  '灾民',
  '内容',
  '规定',
  '救助站',
  '人员',
  '食物',
  '住处',
  '医疗',
  '灾民',
  '基本',
  '亲友',
  '住宿',
  '职业',
  '异地',
  '分流',
  '条例',
  '规定',
  '流浪',
  '人员',
  '分类',
  '食宿',
  '亲友',
  '流浪',
  '度日',
  '流浪',
  '人员',
  '流浪',
  '人员',
  '部门',
  '购票',
  '流浪',
  '保护性',
  '流浪',
  '人员',
  '人员',
  '进站',
  '职业',
  '流浪',
  '人员',
  '职业',
  '流浪',
  '人员',
  '部门',
  '异地',
  '分流',
  '流浪',
  '危重病',
  '精神病人',
  '患病',
  '流浪',
  '人员',
  '原则',
  '人道主义',
  '民政',
  '城管',
  '公安',
  '定点医院',
  '流浪',
  '人员',
  '流浪',
  '人员',
  '公安部门',
  '依法',
  '无法',
  '人员',
  '身份',
  '核实',
  '异地',
  '家属',
  '抚慰金',
  '遇难者',
  '条例',
  '规定',
  '灾害',
  '物资',
  '基本',
  '住所',
  '住宿',
  '医疗

In [24]:
# TF-IDF 算法
def tf(filter_sent):
    # 统计TF值
    tf_dict = {}
    for word in filter_sent:
        if word not in tf_dict:
            tf_dict[word] = 1
        else:
            tf_dict[word] += 1
    for word in tf_dict:
        tf_dict[word] = tf_dict[word] / len(filter_sent)
    return tf_dict
def idf(documents):
    # 统计IDF值
    idf_dict = {}
    for doc in documents:
        for word in set(doc):
            if word not in idf_dict:
                idf_dict[word] = 1
            else:
                idf_dict[word] += 1
    return idf_dict
def tf_idf(documents):
    # 计算TF-IDF值
    idf_dict=idf(documents)
    result = []
    print('idf词典中词的个数为：',len(idf_dict))
    i = 1
    for filter_sent in documents:
        tf_idf_dict = {}
        tf_dict=tf(filter_sent)
        for word in tf_dict:
            tf_idf_dict[word] = tf_dict[word] * math.log(len(documents) / (idf_dict[word]))
        # 提取前10个关键词
        topk = 10
        sort_kwords=sorted(tf_idf_dict.items(), key=operator.itemgetter(1),reverse=True)
        kword_list=[kword for kword,value in sort_kwords[:topk]]
        result.append(sort_kwords)
        print('第%d句用TF-IDF模型结果：'%i,'/ '.join(kword_list))
        i+=1
    return result
# tf_idf(documents)

In [34]:
data = tf_idf(documents)
(
    WordCloud()
        .add(series_name='TF-IDF',
             data_pair=data[-3],    # 要将哪一句话生成词云
             word_size_range=[16, 66],  # 字体大小
             mask_image='.//img.png'  # 在这里设置背景的路径
             )
        .set_global_opts(
        title_opts=options.TitleOpts(
            pos_left='40%',
            title='TF-IDF',  # 词云图的名字
            title_textstyle_opts=options.TextStyleOpts(font_size=33)
        ),
        tooltip_opts=options.TooltipOpts(is_show=True),
    )
        .render('TF-IDF_Vision.html')
)

idf词典中词的个数为： 2769
第1句用TF-IDF模型结果： 童童/ 肛门/ 心脏病/ 先天性/ 手术/ 酒店/ 后门/ 外表/ 常人/ 被遗弃
第2句用TF-IDF模型结果： 流浪/ 人员/ 灾民/ 条例/ 住宿/ 本市/ 基本/ 居民/ 职业/ 异地
第3句用TF-IDF模型结果： 地铁/ 坐票/ 座位/ 车厢/ 行李/ 老街/ 轨道交通/ 赵鹏林/ 大件/ 机场
第4句用TF-IDF模型结果： 市场/ 监管局/ 食品/ 食用油/ 地沟油/ 办法/ 餐厨/ 垃圾处理/ 企业/ 监管部门
第5句用TF-IDF模型结果： 鞋子/ 孩子/ 马学良/ 妹妹/ 运动鞋/ 团县委/ 老师/ 爷爷/ 妈妈/ 学校
第6句用TF-IDF模型结果： 利益/ 志愿/ 志愿者/ 部门/ 长效机制/ 经历/ 研究生/ 变味/ 激励机制/ 令人
第7句用TF-IDF模型结果： 公益活动/ 总支出/ 孩子/ 总结/ 善款
第8句用TF-IDF模型结果： 传统/ 慈善/ 心被/ 初级阶段/ 行政化/ 色彩/ 经验/ 境界/ 符号/ 五彩
第9句用TF-IDF模型结果： 汽车/ 文明/ 作品名称/ 事故/ 作者/ 情景/ 短片/ 驾车/ 中山大学/ 女生
第10句用TF-IDF模型结果： 女性/ 妇女/ 公正/ 性别/ 联合国/ 报告/ 旗舰/ 男性/ 宪法/ 场合
第11句用TF-IDF模型结果： 车门/ 汽车/ 司机/ 郑州/ 车窗/ 水位/ 侧窗/ 车厢/ 水门/ 时应
第12句用TF-IDF模型结果： 弱势/ 儿童/ 黄英男/ 人人/ 彩虹/ 计划/ 题材/ 电影/ 深度/ 种类
第13句用TF-IDF模型结果： 山路/ 小康/ 代步/ 孩子/ 祖国/ 土墙/ 茅屋/ 手脚/ 冻疮/ 身子
第14句用TF-IDF模型结果： 市场开发部/ 袁斌/ 搜狐公司/ 首席/ 财务/ 余楚媛/ 部长/ 北京
第15句用TF-IDF模型结果： 房屋/ 地震/ 新源县/ 和静县/ 有序/ 新疆维吾尔自治区/ 伊犁哈萨克自治州/ 昭苏县/ 特克斯县/ 察布查尔锡伯自治县
第16句用TF-IDF模型结果： 资产/ 黄怒波/ 官办/ 计划/ 信任/ 商铺/ 过户/ 双方/ 慈善机构/ 办理
第17句用TF-IDF模型结果： 咨询电话/ 报名表/ 邮箱/ 项目/ 方案/ 办法/ 资助/ 方式
第18句用

'D:\\2020185 _and_10208\\大三上\\NLP\\实验一：关键词抽取数据集\\TF-IDF_Vision.html'

# TextRank算法

In [36]:
def TextRand():
    keywords = []
    for i in text:
        keywords.append(jieba.analyse.extract_tags(sentence=i,topK=10,allowPOS=('n','ns','nr','nt','nz'),withWeight=True))
    return keywords
TextRand()

[[('童童', 2.3436615837483874),
  ('肛门', 0.5689853733541936),
  ('先天性', 0.5670122492658065),
  ('心脏病', 0.5321411917232258),
  ('手术', 0.45759630376000004),
  ('慈善机构', 0.361052498416129),
  ('被遗弃', 0.3278389688290323),
  ('思源', 0.30705325862258065),
  ('康复', 0.28143638449677416),
  ('公益', 0.28071550389387095)],
 [('流浪', 0.7211609480638607),
  ('人员', 0.4954051302775949),
  ('灾民', 0.37114721266822787),
  ('条例', 0.2798658237056962),
  ('住宿', 0.21975498295949367),
  ('本市', 0.21446410184860762),
  ('异地', 0.16698858565974684),
  ('抚慰金', 0.15558531315822785),
  ('基本', 0.15193535174367087),
  ('居民', 0.14756543043974685)],
 [('地铁', 0.6915860066280723),
  ('坐票', 0.616967005460241),
  ('车厢', 0.4099783183431325),
  ('座位', 0.3825107618486747),
  ('老街', 0.3819930051253012),
  ('赵鹏林', 0.2880666868168675),
  ('行李', 0.27646170248710844),
  ('大件', 0.24069047341084338),
  ('档次', 0.21206774034096385),
  ('乘客', 0.19233213970361446)],
 [('地沟油', 0.6567249284409449),
  ('餐厨', 0.4706601379094488),
  ('监管局', 0.4640

In [41]:
data = TextRand()
(
    WordCloud()
        .add(series_name='TextRand',
             data_pair=data[-1],
             word_size_range=[16, 66],
             # shape='star'
             mask_image='.///moon.jpg'
             )
        .set_global_opts(
        title_opts=options.TitleOpts(
            pos_left='40%',
            title='TextRand',
            title_textstyle_opts=options.TextStyleOpts(font_size=33)
        ),
        tooltip_opts=options.TooltipOpts(is_show=True),
    )
        .render('TextRand_Vision.html')
)

'D:\\2020185 _and_10208\\大三上\\NLP\\实验一：关键词抽取数据集\\TextRand_Vision.html'